In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [166]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X = X_st

In [167]:
X

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  


In [52]:
def calc_logloss(y, y_pred):
    
    y_pred = y_pred[y_pred>0]
    
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [8]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [143]:
def eval_model(X, y, iterations, alpha=1e-2):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        err_.append(err)
        if i % (iterations / 25) == 0:
            print(i, W, err)
            
    return W

In [175]:
W = eval_model(X_st, y, iterations=5000, alpha=2)

200 [-8.90880758 -3.4047485  -1.47147846  7.84969915] 1.0606247109635314
400 [-12.04457679  -1.52428767  -2.55660064  10.36287629] 0.25977592487652046
600 [-14.22318387  -1.68291182  -3.25613011  12.0222098 ] 0.23158929194777828
800 [-16.12009743  -1.83647323  -3.85324671  13.47565125] 0.21146369916432492
1000 [-17.81012201  -2.00397277  -4.37511232  14.7814083 ] 0.1974119749794952
1200 [-19.33193732  -2.13675657  -4.82928098  16.01532088] 0.18724361607195825
1400 [-20.73853815  -2.2595764   -5.24640339  17.15660205] 0.17855923905371557
1600 [-22.04668785  -2.37423804  -5.63247198  18.2177413 ] 0.17105353218939895
1800 [-23.26945132  -2.4817194   -5.99186717  19.2092677 ] 0.16450049004363576
2000 [-24.41748882  -2.58284232  -6.32811374  20.13980869] 0.158728066474803
2200 [-25.49963926  -2.67830294  -6.64410004  21.01653581] 0.15360278785279902
2400 [-26.52331757  -2.76869496  -6.94222452  21.84547489] 0.14901953813547936
2600 [-27.49480249  -2.85452782  -7.22450011  22.63173396] 0.144

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).


In [188]:
def calc_pred_proba(W, X): 
    
    W = W.reshape(X.shape[0], 1)
    z = np.dot(W.T, X)
    A = sigmoid(z)
    
    class_1 = A[A>0.5]
    return class_1

In [189]:
calc_pred_proba(W, X)

array([1.        , 0.93840109, 1.        , 0.5934771 , 0.99999981])


4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [198]:
def calc_pred(W, X):    
    # предополжим что порог у нас 0.5
    pred = []
    
    W = W.reshape(X.shape[0], 1)
    z = np.dot(W.T, X)
    A = sigmoid(z)
    A = A[0]
    for i in range(X.shape[1]):
        if A[i] < 0.5:
            pred.append(0)
        else:
            pred.append(1)
    return pred

In [205]:
y_pred = calc_pred(W, X)
y_pred

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1]

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [259]:
def metrics(y, y_pred):
    accuracy = np.sum(y==y_pred)/len(y)
    print(f'Accuracy = {accuracy}')
    
    TP, FP, FN, TN = 0, 0, 0, 0
    
    for i, _ in enumerate(y):
        if y[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y[i] == 0 and y_pred[i] == 1:
            FP += 1
        if y[i] == 1 and y_pred[i] == 0:
            FN += 1
        if y[i] == 0 and y_pred[i] == 0:
            TN += 1
        
    print()
    print('     : 1 : 0  ')
    print(f'y = 1: {TP} : {FP}')
    print(f'y = 0: {FN} : {TN}')
    
    precision = TP / (TP+FP)
    recall = TP / (TP+FN)
    print()
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    
    f1 = 2*precision*recall/(precision+recall)
    print()
    print(f'F1-score: {f1}')
    
    
metrics(y, y_pred)           

Accuracy = 1.0

     : 1 : 0  
y = 1: 5 : 0
y = 0: 0 : 5

Precision: 1.0
Recall: 1.0

F1-score: 1.0


6. Могла ли модель переобучиться? Почему?

Скорее всего модель переобучена, так как на тестовых данных ее точность 100%, при этом выборка содержит всего 10 наблюдений, что говорит о том, что веса настроены идеально для небольшого конкретного набора данных.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [279]:
def eval_model_l1(X, y, iterations, alpha=1e-2, lmbd = 1e-6):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))+ lmbd / 2 * W / np.abs(W)
        err_.append(err)
        if i % (iterations / 25) == 0:
            print(i, W, err)
            
    return W
eval_model_l1(X_st, y, iterations=5000, alpha=2, lmbd = 1e-4)

200 [-8.9996714  -2.25965726 -0.59398151  9.3503135 ] 0.3357236193190717
400 [-12.04495636  -1.52430917  -2.55659317  10.36325194] 0.2597848713761246
600 [-14.21172272  -1.68201202  -3.25236182  12.01353081] 0.23173368723708182
800 [-16.09800683  -1.83447504  -3.8462142   13.45884199] 0.21168466192209173
1000 [-17.77860774  -2.00098747  -4.36552478  14.755996  ] 0.19763121930720803
1200 [-19.2909673   -2.1331295   -4.81699687  15.98198492] 0.18750659015559276
1400 [-20.68848861  -2.25513223  -5.23149598  17.11589765] 0.17885708568112435
1600 [-21.98793417  -2.36900824  -5.61506573  18.16998141] 0.17137945182984118
1800 [-23.20229518  -2.47573017  -5.97206057  19.15470522] 0.16484937607825167
2000 [-24.34217299  -2.57611551  -6.30598488  20.07864852] 0.15909604079030315
2200 [-25.4163593   -2.67085688  -6.61971088  20.9489444 ] 0.1539868746041989
2400 [-26.43223109  -2.7605451   -6.91562424  21.7715878 ] 0.14941743135980207
2600 [-27.39603642  -2.84568728  -7.1957276   22.55166184] 0.14

array([-36.29094951,  -3.62875482,  -9.76048417,  29.7251916 ])

In [277]:
def eval_model_l2(X, y, iterations, alpha=1e-2, lmbd = 1e-6):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))+ lmbd * W
        err_.append(err)
        if i % (iterations / 25) == 0:
            print(i, W, err)
            
    return W
eval_model_l2(X_st, y, iterations=5000, alpha=2, lmbd = 1e-4)

200 [-8.83493367 -3.10770754 -1.58760879  7.43191534] 1.2308634499109457
400 [-11.80107755  -1.50662973  -2.47892136  10.17814456] 0.2634720954803219
600 [-13.77213911  -1.64763098  -3.11443329  11.67615392] 0.23703200937923236
800 [-15.44126643  -1.77787112  -3.64235922  12.95429185] 0.21829358531378115
1000 [-16.89177752  -1.90677351  -4.09544557  14.06404648] 0.2044130964787029
1200 [-18.16251873  -2.03416394  -4.48219366  15.06473771] 0.19497529891765286
1400 [-19.29594961  -2.13264981  -4.82003741  15.98436766] 0.18746616036557043
1600 [-20.32283428  -2.2222222   -5.12482909  16.81751936] 0.18104907098457088
1800 [-21.25727969  -2.30396939  -5.40120012  17.57555512] 0.17551097925484505
2000 [-22.11078304  -2.37880431  -5.65286664  18.26778441] 0.17069088862085519
2200 [-22.89288447  -2.44749968  -5.88287655  18.90195057] 0.16646457181206784
2400 [-23.6116037   -2.51071507  -6.09376642  19.48457256] 0.1627345775952216
2600 [-24.27375226  -2.56901752  -6.28767181  20.02119257] 0.159

array([-29.34165703,  -3.01623298,  -7.76148009,  24.12237818])